In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [2]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.shape[0]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


968557

In [3]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [4]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    67317
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
LA    13668
OR    13456
FL    10045
UT     9295
IA     8295
NE     5804
HI     4980
NM     4485
ID     4416
ND     3222
MT     3144
MO     3120
PR     2952
SD     2695
OK     2535
IN     2348
WY     2139
SC     2045
MS     1589
NV     1569
KS     1485
GU      443
RI      414
VI      290
MP       82
NC       11
PA        8
AL        8
AK        5
AZ        4
TN        3
NJ        3
MA        3
IL        2
AR        2
ME        2
WV        1
CT        1
OH        1
GA        1
Name: BorrowerState, dtype: int64

In [8]:
ppp_togeoLA = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "LA"]
ppp_togeoLA.shape[0] #13668

13668

In [9]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoLA_add = ppp_togeoLA[cols]
ppp_togeoLA_add

,LoanNumber,full_add
384714,1115657202,"2400 Veterans Memorial Blvd. Suite 400, KENNER..."
384715,1623197104,"400 Poydras Street Ste 1800 0.0, New Orleans, LA"
384716,4241517309,"1515 River Oaks Road East, harahan, LA"
384717,5102977301,"17630 Perkins Rd, BATON ROUGE, LA"
384718,5378948703,"1307 Oretha Castle Haley Blvd Ste 302, New Orl..."
...,...,...
398377,9651717708,"1000 Chinaberry Dr, Bossier City, LA"
398378,9659377206,"1440 N. Dorgenois St, NEW ORLEANS, LA"
398379,9719897702,"20252 HIGHWAY 36, COVINGTON, LA"
398380,9806418300,"204 Cresthill Dr, Youngsville, LA"


In [10]:
add = ppp_togeoLA["full_add"].astype(str).to_list()

In [ ]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('22')] #LA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1

In [14]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

13668
13698
13668
13668


<b>Add remaining values (2801) to result_df(5461)

In [ ]:
pd.set_option("display.max_rows", None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df_drop = lat_df.drop(index=[3072,6438,2036,4030,8660,9905,9326,7131,8122,8528,1523,2325,1207,1301,
12950,10750,3697,13333,8979,10526,11714,195,11759,11199,13279,1676,5106,4574,4576,7891,167,9705], axis=1)
lat_df_drop = lat_df_drop.reset_index()
lat_df_drop = lat_df_drop["Lat"]
lat_df_drop

In [ ]:
line = pd.DataFrame({"Lat": "30.207064"}, index=[2029.5])
df2 = lat_df_drop.append(line, ignore_index=False)
df2 = df2.sort_index().reset_index(drop=True)
df2["Lat"] = df2["Lat"].fillna(df2[0])
df2 = df2["Lat"]
df2

In [ ]:
line = pd.DataFrame({"Lat": "29.973152"}, index=[7116.5])
df3 = df2.append(line, ignore_index=False)
df3 = df3.sort_index().reset_index(drop=True)
df3["Lat"] = df3["Lat"].fillna(df3[0])
df3 = df3["Lat"]
df3

In [46]:
lat_list = df3.to_list()

In [ ]:
result_df = pd.DataFrame({ "full_add":add, "Lat":lat_list, "Long":long, "FIPS_z": geo_id})
result_df 

In [ ]:
df3["full_add"] = add_df_list
df3

In [42]:
result_df = df3

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [49]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoLA.Lat = ppp_togeoLA.Lat.fillna(ppp_togeoLA.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoLA.Long = ppp_togeoLA.Long.fillna(ppp_togeoLA.full_add.map(n))
o = dict(zip(result_df.full_add,result_df.FIPS_z))
ppp_togeoLA.FIPS_z = ppp_togeoLA.FIPS_z.fillna(ppp_togeoLA.full_add.map(o))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [51]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoLA_parsed = ppp_togeoLA[cols]
ppp_togeoLA_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
384714,1115657202,220510205062018,30.006386,-90.23586
384715,1623197104,220710134004025,29.948801,-90.06754
384716,4241517309,220510242023039,29.961603,-90.20082
384717,5102977301,220330040212030,30.346773,-91.03166
384718,5378948703,None,None,None


In [52]:
ppp_geo = ppp_geo.merge(ppp_togeoLA_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.369867,NaN,NaN,NaN,NaN
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.066165,NaN,NaN,NaN,NaN
2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.302559,NaN,NaN,NaN,NaN
3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.372550,NaN,NaN,NaN,NaN
4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.913680,NaN,NaN,NaN,NaN


In [53]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [54]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [55]:
ppp_geo_la = ppp_geo.loc[ppp_geo["BorrowerState"] =="LA"]
ppp_geo_la.shape[0]

13668

In [57]:
null_check = ppp_geo_la.loc[ppp_geo_la["FIPS_z"].isnull()]
null_check.shape[0]

2760